> ### EEE2020: Data Structures & Algorithms

# Assignment \# VI: Graphs

<div style="text-align: right"> **Due date: June 21, 2021.** </div> 
<div style="text-align: right"> **Please upload your data @ LearnUs by 7 PM.** </div> 
<br>
<div style="text-align: right"> Daesung Jeon(전대성): <a href="mailto:ds.jeon@yonsei.ac.kr">ds.jeon@yonsei.ac.kr</a> 
<div style="text-align: right"> Hyekang Park(박혜강): <a href="mailto:hyekang.park@yonsei.ac.kr">hyekang.park@yonsei.ac.kr</a> </div>
</div>

-------------------------------


### *Assignment Instructions:*
1. Write a program implementing a particular algorithm to solve a given problem.
2. You can use both Korean and Engligh for your report.
3. **Analyze the algorithm theoretically and empirically, and write discussion.** 
4. **Report your results.**  
    - You must do assignments and labs yourself. 
    - <span style="color:red">**Do not copy any part of your friends’ code.**</span>
    - Otherwise, you will get a F grade.

-----------------

<h2><span style="color:blue">[2016145101] [안장환]</span> </h2>

In [289]:
import datetime
print("This code is written at " + str(datetime.datetime.now()))

This code is written at 2021-06-21 18:49:09.287953


-------------------------------

In [290]:
################ DO NOT modify this code ######################
import sys

class Vertex:
    def __init__(self,key):
        self.id = key
        self.connectedTo = {}
        self.color = 'white'
        self.dist = sys.maxsize
        self.pred = None
        self.disc = 0
        self.fin = 0
    
    def addNeighbor(self,nbr,weight=0):
        self.connectedTo[nbr] = weight
    
    def removeNeighbor(self,nbr):
        self.connectedTo.pop(nbr, None)
    
    def setColor(self,color):
        self.color = color
        
    def setDistance(self,d):
        self.dist = d

    def setPred(self,p):
        self.pred = p

    def setDiscovery(self,dtime):
        self.disc = dtime
        
    def setFinish(self,ftime):
        self.fin = ftime
        
    def getFinish(self):
        return self.fin
        
    def getDiscovery(self):
        return self.disc
        
    def getPred(self):
        return self.pred
        
    def getDistance(self):
        return self.dist
        
    def getColor(self):
        return self.color
    
    def getConnections(self):
        return self.connectedTo.keys()

    def getId(self):
        return self.id
    
    def getWeight(self,nbr):
        return self.connectedTo[nbr]

    def __str__(self):
        return str(self.id) + ' connectedTo: ' + str([x.id for x in self.connectedTo])
    
class Graph:
    def __init__(self):
        self.vertList = {}
        self.numVertices = 0

    def addVertex(self,key):
        self.numVertices = self.numVertices + 1
        newVertex = Vertex(key)
        self.vertList[key] = newVertex
        return newVertex

    def getVertex(self,n):
        if n in self.vertList:
            return self.vertList[n]
        else:
            return None

    def __contains__(self,n):
        return n in self.vertList

    def addEdge(self,f,t,weight=0):
        if f not in self.vertList:
            nv = self.addVertex(f)
        if t not in self.vertList:
            nv = self.addVertex(t)
        self.vertList[f].addNeighbor(self.vertList[t], weight)

    def getVertices(self):
        return self.vertList.keys()

    def __iter__(self):
        return iter(self.vertList.values())

class PriorityQueue:
    def __init__(self):
        self.heapArray = [(0,0)]
        self.currentSize = 0

    def buildHeap(self,alist):
        self.currentSize = len(alist)
        self.heapArray = [(0,0)]
        for i in alist:
            self.heapArray.append(i)
        i = len(alist) // 2            
        while (i > 0):
            self.percDown(i)
            i = i - 1
                        
    def percDown(self,i):
        while (i * 2) <= self.currentSize:
            mc = self.minChild(i)
            if self.heapArray[i][0] > self.heapArray[mc][0]:
                tmp = self.heapArray[i]
                self.heapArray[i] = self.heapArray[mc]
                self.heapArray[mc] = tmp
            i = mc
                
    def minChild(self,i):
        if i*2 > self.currentSize:
            return -1
        else:
            if i*2 + 1 > self.currentSize:
                return i*2
            else:
                if self.heapArray[i*2][0] < self.heapArray[i*2+1][0]:
                    return i*2
                else:
                    return i*2+1

    def percUp(self,i):
        while i // 2 > 0:
            if self.heapArray[i][0] < self.heapArray[i//2][0]:
                tmp = self.heapArray[i//2]
                self.heapArray[i//2] = self.heapArray[i]
                self.heapArray[i] = tmp
            i = i//2
 
    def add(self,k):
        self.heapArray.append(k)
        self.currentSize = self.currentSize + 1
        self.percUp(self.currentSize)

    def delMin(self):
        retval = self.heapArray[1][1]
        self.heapArray[1] = self.heapArray[self.currentSize]
        self.currentSize = self.currentSize - 1
        self.heapArray.pop()
        self.percDown(1)
        return retval
        
    def isEmpty(self):
        if self.currentSize == 0:
            return True
        else:
            return False

    def decreaseKey(self,val,amt):
        done = False
        i = 1
        myKey = 0
        while not done and i <= self.currentSize:
            if self.heapArray[i][1] == val:
                done = True
                myKey = i
            else: 
                i = i + 1
        if myKey > 0:
            self.heapArray[myKey] = (amt,self.heapArray[myKey][1])
            self.percUp(myKey)
            
    def __contains__(self,vtx):
        for pair in self.heapArray:
            if pair[1] == vtx:
                return True
        return False

### PROBLEM I - SCCs & DFS (40점)

#### 1. Kosaraju’s Algorithm (20점)
**Directed graph or Undirected graph**에서 **SCCs(Strongly Connected Compents)** 찾는 **Kosaraju's algorithm**을 구현하려 한다. 주어진 class ```DFSGraph```에 대하여 다음 method를 완성하시오.
> * ```def getTransposition(self)```: 주어진 방향그래프에 대해 역방향을 가지는 ```DFSGraph```객체를 반환한다.
> * ```def getSCCs(self)```: Kosaraju's algorithm을 이용하여 strongly connected compents를 **nested list**의 형태로 반환한다. 

#### 2. Detect cycle (20점)
**Cycle**이란 하나의 vertex *u*에서 시작해 어떠한 경로를 지나 다시 그 vertex *u*로 돌아오는 경로를 말한다. **Directed graph or Undirected graph**에서 **DFS**를 이용하여 cycle을 찾으려고 한다. 주어진 class ```DFSGraph```에 대하여 다음 method를 완성하시오.
> * ```def IsCycle(self)```: Cycle이 있는 경우 ```True```를, 아니면 ```False```를 반환한다.

#### HINT 
- ```def dfsvisit(self,startVertex, stack = None)```을 활용한다.




#### NOTE
- 강의자료를 활용할 수 있다.
- 외부 라이브러리는 사용하지 않는다(사용 시 해당 문항 0점).
- ```###CODE HERE###``` 이외의 코드를 수정하지 않는다.
- ```###CODE HERE###``` 중 설명이 필요한 부분에 주석을 단다. 
- Discussion에 알고리즘이 어떻게 동작하는지 설명한다.

In [291]:
class DFSGraph(Graph):
    def __init__(self):
        super().__init__()
        self.time = 0
        self.isCycle = False

    def dfs(self):
        for aVertex in self:
            aVertex.setColor('white')
            aVertex.setPred(-1)
        for aVertex in self:
            if aVertex.getColor() == 'white':
                self.dfsvisit(aVertex)

    def dfsvisit(self,startVertex, stack = None):
        ### CODE HERE ###
        
        startVertex.setColor('gray')
        self.time += 1
        startVertex.setDiscovery(self.time)
        for nextVertex in startVertex.getConnections():
            if nextVertex.getColor()=='gray' :
                if type(self)==type(DFSGraph()):
                    self.isCycle=True
                elif type(self)==type(Undirected_DFSGraph()) and startVertex:
                    if (startVertex.getDiscovery()-nextVertex.getDiscovery())!=1:
                        self.isCycle=True
            if nextVertex.getColor() == 'white':
                nextVertex.setPred(startVertex)
                self.dfsvisit(nextVertex,stack)
        startVertex.setColor('black')
        self.time += 1
        startVertex.setFinish(self.time)
        if stack!=None:
            stack.append(startVertex)
        #################
        
        
    def print_time(self):
        for key in sorted(list(self.getVertices())):
            print(str(key) + ": " + str(self.getVertex(key).getDiscovery()) + "/" + str(self.getVertex(key).getFinish()))
            
    def getTransposition(self):
        ### CODE HERE ###
        new_DFSGraph=DFSGraph()
        
        for vertex in self:
            for connected in vertex.connectedTo:
                new_DFSGraph.addEdge(connected.id,vertex.id)
         
        return new_DFSGraph
        #################
    
    def getSCCs(self):
        ## CODE HERE ###
        stack=[]
        scc_list=[]
        trans_graph=self.getTransposition()
        
        for aVertex in self:
            aVertex.setColor('white')
            aVertex.setPred(-1)
        for aVertex in self:
            if aVertex.getColor() == 'white':
                self.dfsvisit(aVertex,stack)
#         print(stack)
        
        for aVertex in trans_graph:
            aVertex.setColor('white')
        
        while stack:
            last_fin_id=stack.pop().id
            last_fin=trans_graph.getVertex(last_fin_id)
            if last_fin.getColor()=='white':
                if last_fin.getConnections() and last_fin.getColor()=='white':
                    sub_list=[]
                    trans_graph.dfsvisit(last_fin,sub_list)
                    sub_list=[vertex.id for vertex in sub_list]
                else:
                    last_fin.setColor('black')
                    sub_list=[]
                    sub_list.append(last_fin.id)
                scc_list.append(sub_list)

        return scc_list
            
            
        ################
    
    def IsCycle(self):
        ### CODE HERE ###

        for aVertex in self:
            aVertex.setColor('white')
        for aVertex in self:
            if aVertex.getColor() == 'white':
                self.dfsvisit(aVertex)
        return self.isCycle
#             if aVertex.getColor() == 'white':
#                 aVertex.setColor('gray')
#                 for nextVertex in aVertex.getConnections(): 
#                     if nextVertex.getColor() == 'gray' and (nextVertex.id==aVertex.id):
#                         return True
#                     elif nextVertex.getColor()=='white':
#                         self.dfsvisit(nextVertex)

#             aVertex.setColor('black')            
            
        #################

In [292]:
graph = DFSGraph()

myVertices = ['A','B','C','D','E','F']
# add vertices
for i in range(len(myVertices)):
    graph.addVertex(myVertices[i])

graph.addEdge('A', 'B')
graph.addEdge('A', 'D')
graph.addEdge('B', 'C')
graph.addEdge('B', 'D')
graph.addEdge('D', 'E')
graph.addEdge('E', 'B')
graph.addEdge('E', 'F')
graph.addEdge('F', 'C')

print(graph.getSCCs())
print(graph.IsCycle())

[['A'], ['D', 'E', 'B'], ['F'], ['C']]
True


### Describe and Discuss your answer

- getTransposition: 새로운 DFSGraph 객체를 정의해준다. 그리고 기존 DFSGraph 객체의 edge에서 vertex의 from, to 위치를 바꾸어 DFSGraph에 addEdge준다. 새로운 DFSGraph객체를 return 하여 기존 graph에서 transposed 된 graph를 얻는다.
- getSCCs: dfsvisit method를 통해 finish time이 작은 순서로 stack에 쌓는다. stack.pop()을 startVertex로 이용하여 getTransposition을 통해 얻는 graph에서 dfsvisit을 실행한다. DFS를 통해 찾은 vertex를 sub_list에 append하고, 이 sub_list를 다시 scc_list에 append해주면 strongly connected components를 구할 수 있다.
- dfsvisit : startVertex부터 시작하여 연결된 vertex의 color가 white이면 stack개념을 활용하여 recursive하게 dfsvisit을 실행한다. 탐색한 vertex가 gray일 때, directed graph에서는 cycle이 존재함을 의미한다. 그리고 undirected에서는 3개 이상의 vertex에 대해서만 cycle이 정의되므로, gray인 vertex가 연속으로 탐색된 vertex가 아니라면(discovery time의 차이가 1이 아니라면) cycle이라고 판단할 수 있다.
- IsCycle: graph의 각 vertex에 대해 dfsvisit을 실행하고 self.isCycle attribute를 return한다.

___

### PROBLEM II - Minimum Spanning Trees (35점)
#### 1. Undirected Graph (5점)
주어진 class ```Undirected_DFSGraph(DFSGraph)```에 대해서 **PROBLEM I**에서 구현한 ```getSCCs(self)```, ```IsCycle(self)```이 동작하는지 확인하시오. 

#### 2. Kruskal algorithm & Prim algorithm (30점)
주어진 **Undirected Graph**에 대해서 **Minimum Spanning Trees**을 구하는 ***Kruskal algorithm***과 ***Prim algorithm***을 구현하려 한다. 다음 조건에 맞는 함수들을 구현하시오.  
* ***Kruskal algorithm***
> * ```def kruskal_cluster(graph)```, ```def kruskal_cycle(graph)```: ***Kruskal algorithm***을 이용하여, 입력 받은 ```Undirected_DFSGraph``` 객체 내의 **MST**를 찾은 후 ```Undirected_DFSGraph``` 객체 형태로 반환한다. 
> * ```def kruskal_cluster(graph)```: 강의자료의 **Pseudo-Code**를 참조하여 구현하시오.
> * ```def kruskal_cycle(graph)```: ```IsCycle(self)```을 이용해 ***Kruskal algorithm***을 구현하시오. 서로 다른 두 node가 같은 cluster에 존재하는지 여부는 graph의 cycle을 이용해 판단 가능함. 즉, 같은 cluster에 속한 node들을 연결하는 edge는 cycle을 형성하므로, 해당 edge는 **MST**에서 제외해야함. 사이클 생성 여부는 ```IsCycle(self)```으로 판단 가능함.   
> * 주어진 ```PriorityQueue``` 객체를 이용한다.
* ***Prim algorithm***
> * ```def prim(G,start)```: ***Prim algorithm***을 이용하여, 입력받은  start_vertex  부터  ```Undirected_DFSGraph``` 객체  내의 **MST**를 찾은 뒤 ```Undirected_DFSGraph``` 객체 형태로 반환한다.  
> * 주어진 ```PriorityQueue``` 객체를 이용한다.
#### NOTE
- 강의자료를 활용할 수 있다.
- 외부 라이브러리는 사용하지 않는다(사용 시 해당 문항 0점).
- ```###CODE HERE###``` 이외의 코드를 수정하지 않는다.
- ```###CODE HERE###``` 중 설명이 필요한 부분에 주석을 단다. 
- Discussion에 알고리즘이 어떻게 동작하는지 설명하고 ```def kruskal_cluster(graph)```와 ```def kruskal_cycle(graph)```의 complexity를 분석한다.

In [293]:
class Undirected_DFSGraph(DFSGraph):
    def __init__(self):
        super().__init__()

    def addEdge(self,f,t,weight=0):
        if f not in self.vertList:
            nv = self.addVertex(f)
        if t not in self.vertList:
            nv = self.addVertex(t)
        self.vertList[f].addNeighbor(self.vertList[t], weight)
        self.vertList[t].addNeighbor(self.vertList[f], weight)
        
    def removeEdge(self,f,t,weight=0):
        self.vertList[f].removeNeighbor(self.vertList[t])
        self.vertList[t].removeNeighbor(self.vertList[f])

In [294]:
graph = Undirected_DFSGraph()

myVertices = ['A','B','C','D','E','F']
# add vertices
for i in range(len(myVertices)):
    graph.addVertex(myVertices[i])

graph.addEdge('A', 'B')
graph.addEdge('B', 'C')
graph.addEdge('C', 'A')

graph.addEdge('D', 'E')
graph.addEdge('E', 'F')
graph.addEdge('F', 'D')

print(graph.getSCCs())
print(graph.IsCycle())

[['F', 'E', 'D'], ['C', 'B', 'A']]
True


In [295]:
graph = Undirected_DFSGraph()

myVertices = ['A','B','C','D','E','F']
# add vertices
for i in range(len(myVertices)):
    graph.addVertex(myVertices[i])

graph.addEdge('A', 'B')
graph.addEdge('B', 'C')

graph.addEdge('C', 'D')
graph.addEdge('D', 'E')
graph.addEdge('E', 'F')


print(graph.getSCCs())
print(graph.IsCycle())

[['F', 'E', 'D', 'C', 'B', 'A']]
False


In [296]:
def kruskal_cycle(graph):
    ### CODE HERE ###
    mstGraph=Undirected_DFSGraph()
    edge_count=0
 
    for v in graph:
        mstGraph.addVertex(v.id)
    
    pq=PriorityQueue()
    heaplist=[]
    
    for v in graph:
        for x in v.connectedTo:
            heaplist.append((v.getWeight(x),(v.id,x.id)))
    pq.buildHeap(heaplist)
    while edge_count< 2*mstGraph.numVertices-1:
        weight=pq.heapArray[1][0]
        edge=pq.delMin()
        mstGraph.addEdge(edge[0],edge[1],weight)
        if mstGraph.IsCycle():
            mstGraph.removeEdge(edge[0],edge[1],weight)
        else:
            edge_count+=1
    return mstGraph    
    #################

def kruskal_cluster(graph):
    ### CODE HERE ###
    mstGraph=Undirected_DFSGraph()
    edge_count=0
    cluster={}
    i=0
    for v in graph:
        mstGraph.addVertex(v.id)
        cluster[v.id]=i
        i+=1
    
    pq=PriorityQueue()
    heaplist=[]
    for v in graph:
        for x in v.connectedTo:
            heaplist.append((v.getWeight(x),(v.id,x.id)))
    pq.buildHeap(heaplist)

    while edge_count< mstGraph.numVertices-1:
        weight=pq.heapArray[1][0]
        edge=pq.delMin()
        if cluster[edge[0]]!=cluster[edge[1]]:
            if cluster[edge[0]]>cluster[edge[1]]:
                cluster[edge[0]]=cluster[edge[1]]
            else:
                cluster[edge[1]]=cluster[edge[0]]
            mstGraph.addEdge(edge[0],edge[1],weight)
            edge_count+=1
    
    return mstGraph      

    #################

def prim(G,start):
    ### CODE HERE ###
    mstGraph=Undirected_DFSGraph()
    start=G.getVertex(start)
    pq = PriorityQueue()
    
    for v in G:
        v.setDistance(sys.maxsize)
        v.setPred(None)
    start.setDistance(0)
    pq.buildHeap([(v.getDistance(),v) for v in G])
    
    while not pq.isEmpty():
        currentVert = pq.delMin()
        for nextVert in currentVert.getConnections():
            newCost = currentVert.getWeight(nextVert) 
            if nextVert in pq and newCost<nextVert.getDistance():
                if nextVert.getPred():
                    nextVert.removeNeighbor(nextVert.getPred())
                    nextVert.setPred(None)
                else:
                    nextVert.setPred(currentVert)
                    nextVert.setDistance(newCost)
                    pq.decreaseKey(nextVert,newCost)
                mstGraph.addEdge(nextVert.id,currentVert.id,newCost)
                
    return mstGraph
    
    #################

In [297]:
g = Undirected_DFSGraph()

myVertices = ['A','B','C','D','E','F']
# add vertices
for i in range(len(myVertices)):
    graph.addVertex(myVertices[i])

g.addEdge('A', 'B', 1)
g.addEdge('A', 'C', 2)
g.addEdge('B', 'C', 3)
g.addEdge('C', 'D', 4)
g.addEdge('C', 'E', 5)
g.addEdge('C', 'F', 6)
g.addEdge('D', 'E', 7)
g.addEdge('E', 'F', 8)

g_mst2 = kruskal_cluster(g)
print('#' * 10 + 'kruskal_cluster' + '#' * 10)


for alpha in myVertices:
    print(alpha, 'is connected to')
    for k, v in g_mst2.getVertex(alpha).connectedTo.items():
        print(k.getId(), ':', v)
    print('-' * 20)

g_mst3 = prim(g, 'A')
print('#' * 10 + 'prim' + '#' * 10)


for alpha in myVertices:
    print(alpha, 'is connected to')
    for k, v in g_mst3.getVertex(alpha).connectedTo.items():
        print(k.getId(), ':', v)
    print('-' * 20)
    
    
g_mst1 = kruskal_cycle(g)
print('#' * 10 + 'kruskal_cycle' + '#' * 10)

for alpha in myVertices:
    print(alpha, 'is connected to')
    for k, v in g_mst1.getVertex(alpha).connectedTo.items():
        print(k.getId(), ':', v)
    print('-' * 20)



##########kruskal_cluster##########
A is connected to
B : 1
C : 2
--------------------
B is connected to
A : 1
--------------------
C is connected to
A : 2
D : 4
E : 5
F : 6
--------------------
D is connected to
C : 4
--------------------
E is connected to
C : 5
--------------------
F is connected to
C : 6
--------------------
##########prim##########
A is connected to
B : 1
C : 2
--------------------
B is connected to
A : 1
--------------------
C is connected to
A : 2
D : 4
E : 5
F : 6
--------------------
D is connected to
C : 4
--------------------
E is connected to
C : 5
--------------------
F is connected to
C : 6
--------------------


IndexError: list index out of range

### Describe and Discuss your answer

- kruskal_cycle : edge weight가 작은 순서로 edge를 추가하는데 cycle을 형성하지 않는다면 edge를 추가해 vertex를 연결해준다. pq.delMin()에 의해 O(logV), IsCycle에 의해 O(E)가 edge 개수만큼 반복되므로 O(logV)+O(E^2))=O(E^2)가 된다.
- kruskal_cluster: 모든 edge의 (edgeweight, (vertex1, vertex2)) tuple을 정의하여 PriorityQueue를 build해준다. edge의 weight가 작은 순서대로(pq.delMin()) cluster를 연결해주는데, 같은 cluster가 아닐 경우에만 edge를 추가해준다. cluster는 dictionary로 각 vertex의 value를 0부터 부여하고, 연결될 때 두 vertex의 value 중 큰 값은 작은 값으로 변경하여 같은 cluster에 있도록 했다. graph의 edge 수가 graph의 vertex개수보다 하나 작을 때 까지 반복한다. while loop에 의해 pq.delMin() O(logV)가 edge 개수만큼 반복되므로 O(ElogV)가 된다.
- prim : 기존의 prim algorithm에서 새로운 Undirected_DFSGraph를 정의해주고, addEdge(nextVert.id,currentVert.id,newCost)한 뒤에 새로운 Undirected_DFSGraph를 return한다.

___

### PROBLEM III - Shortest Path Problems (25점)
#### 1. Dijkstra algorithm & Bellman-Ford algorithm (25점)
주어진 **Directed Graph**에 대해서 한 정점에서 다른 모든 정점까지의 **Shortest Path**를 구하는 ***Dijkstra algorithm***과 ***Bellman-Ford algorithm***을 구현하려 한다. 다음 조건에 맞는 함수들을 구현하시오.
> * ```def dijkstra(aGraph,start)```: Dijkstra algorithm을 이용하여, 주어진  𝑔𝑟𝑎𝑝ℎ (Graph 객체)의  𝑠𝑡𝑎𝑟𝑡_𝑣𝑒𝑟𝑡𝑒𝑥 (Vertex 객체)에서 출발하여 다른 모든 node까지의 shortest path를 dictionary 형태로 반환한다. 주어진 ```PriorityQueue``` 객체를 이용한다.
> * ```def bellman_ford(graph, s)```:  Bellman-Ford 알고리즘을 이용하여, 주어진  𝑔𝑟𝑎𝑝ℎ (Graph 객체)의  𝑠𝑡𝑎𝑟𝑡_𝑣𝑒𝑟𝑡𝑒𝑥 (Vertex 객체)에서 출발하여 다른 모든 node까지의 shortest path를 dictionary 형태로 반환한다. 단, **negative weight cycle**이 있을 경우 ```False```을 반환한다.

#### NOTE
- 강의자료를 활용할 수 있다.
- 외부 라이브러리는 사용하지 않는다(사용 시 해당 문항 0점).
- ```###CODE HERE###``` 이외의 코드를 수정하지 않는다.
- ```###CODE HERE###``` 중 설명이 필요한 부분에 주석을 단다. 
- Discussion에 알고리즘이 어떻게 동작하는지 설명하고 Complexity를 분석한다.

In [298]:
def dijkstra(aGraph,start):
    ### CODE HERE ###
    distance={}
    pq = PriorityQueue()
    start.setDistance(0)
    pq.buildHeap([(v.getDistance(),v) for v in aGraph])
    while not pq.isEmpty():
        currentVert = pq.delMin()
        for nextVert in currentVert.getConnections():
            newDist = currentVert.getDistance() \
                    + currentVert.getWeight(nextVert)
            if newDist < nextVert.getDistance():
                nextVert.setDistance( newDist )
                nextVert.setPred(currentVert)
                pq.decreaseKey(nextVert,newDist)
   
        if currentVert == start:        
            distance[currentVert]=0
        else:
            dist=0
            vert=currentVert
            distance[vert]=currentVert.getDistance()

    return distance
    #################
    

    
def bellman_ford(graph, s):
    ### CODE HERE ###
    distance={}
    edge_list=[]
    for v in graph:
        v.setPred(None)
        v.setDistance(sys.maxsize)
        for connect in v.getConnections():
            edge_list.append((v, connect))
    s.setDistance(0)
    count=0
    
    for i in range(graph.numVertices):
        for e in edge_list:
            newdist= e[0].getDistance() + e[0].getWeight(e[1])
            if newdist < e[1].getDistance():
                e[1].setDistance(newdist)
                e[1].setPred(e[0])
                
    for e in edge_list:
        newdist= e[0].getDistance() + e[0].getWeight(e[1])
        if newdist < e[1].getDistance():
            return False
    
    for v in graph:
        if v == s:
            distance[v]=0
        else:
            distance[v]=v.getDistance()

    return distance
    #################

In [299]:
graph = Graph()
graph.addVertex('u')
graph.addVertex('v')
graph.addVertex('w')
graph.addVertex('x')
graph.addVertex('y')
graph.addVertex('z')

graph.addEdge('u', 'v', 2)
graph.addEdge('u', 'x', 1)
graph.addEdge('v', 'x', 2)
graph.addEdge('v', 'w', 3)
graph.addEdge('u', 'w', 5)
graph.addEdge('x', 'w', 3)
graph.addEdge('x', 'y', 1)
graph.addEdge('y', 'w', 1)
graph.addEdge('y', 'z', 1)
graph.addEdge('w', 'z', 5)

print('#' * 10 + 'dijkstra' + '#' * 10)
path1 = dijkstra(graph, graph.getVertex('u'))
for v,dist in path1.items():
    print(v.getId(), ':', dist) 
    
print('#' * 10 + 'bellman_ford' + '#' * 10)
path2 = bellman_ford(graph, graph.getVertex('u'))
for v,dist in path2.items():
    print(v.getId(), ':', dist) 

##########dijkstra##########
u : 0
x : 1
v : 2
y : 2
w : 3
z : 3
##########bellman_ford##########
u : 0
v : 2
w : 3
x : 1
y : 2
z : 3


In [300]:
graph = Graph()
graph.addVertex('s')
graph.addVertex('t')
graph.addVertex('x')
graph.addVertex('y')
graph.addVertex('z')

graph.addEdge('t', 'x', 5)
graph.addEdge('t', 'y', 8)
graph.addEdge('t', 'z', -4)
graph.addEdge('x', 't', -2)
graph.addEdge('y', 'x', -3)
graph.addEdge('y', 'z', 9)
graph.addEdge('z', 'x', 7)
graph.addEdge('z', 's', 2)
graph.addEdge('s', 't', 6)
graph.addEdge('s', 'y', 7)

print('#' * 10 + 'bellman_ford' + '#' * 10)
path = bellman_ford(graph, graph.getVertex('s'))
for v,dist in path.items():
    print(v.getId(), ':', dist) 

print('#' * 10 + 'bellman_ford' + '#' * 10)
graph.addEdge('z', 'x', -8)    
print(bellman_ford(graph, graph.getVertex('s')))


##########bellman_ford##########
s : 0
t : 2
x : 4
y : 7
z : -2
##########bellman_ford##########
False


### Describe and Discuss your answer

- dijkstra: PriorityQueue에 각 vertex에 대해 (distance,vertex) tuple을 buildheap해준다. startVertex부터 시작하여 연결된 vertex까지의 거리가 연결된 vertex가 가지고 있는 dist보다 작으면 dist값을 작은 값으로 바꿔준다. pq.delMin()에 의해 O(logV) complexity를 가지고, while loop에 의해 V번 반복되므로 O(VlogV)가 된다. for loop가 각 Edge에 대해 decreaseKey()를 실행하므로 O(ElogV)를 가진다. 따라서 전체적으로 O((V+E)logV)가 된다.
- bellman_ford: 각 edge에 대해 relaxation을 진행하고, 전체적으로 vertex수 만큼 반복한다. vertex 개수에 따른 for문, 그 안에 edge에 따른 for loop를 돌기 때문에 O(VE)가 된다.

___